In [1]:
import asyncio
import os
from typing import Any

from agent_framework._sessions import AgentSession, BaseContextProvider, SessionContext
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

In [2]:
class UserNameProvider(BaseContextProvider):
    """A simple context provider that remembers the user's name."""

    def __init__(self) -> None:
        super().__init__(source_id="user-name-provider")
        self.user_name: str | None = None

    async def before_run(
        self,
        *,
        agent: Any,
        session: AgentSession,
        context: SessionContext,
        state: dict[str, Any],
    ) -> None:
        """Called before each agent invocation — add extra instructions."""
        if self.user_name:
            context.instructions.append(f"The user's name is {self.user_name}. Always address them by name.")
        else:
            context.instructions.append("You don't know the user's name yet. Ask for it politely.")

    async def after_run(
        self,
        *,
        agent: Any,
        session: AgentSession,
        context: SessionContext,
        state: dict[str, Any],
    ) -> None:
        """Called after each agent invocation — extract information."""
        for msg in context.input_messages:
            text = msg.text if hasattr(msg, "text") else ""
            if isinstance(text, str) and "my name is" in text.lower():
                # Simple extraction — production code should use structured extraction
                self.user_name = text.lower().split("my name is")[-1].strip().split()[0].capitalize()

In [3]:
credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    deployment_name=os.environ["AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME"],
    credential=credential,
)

memory = UserNameProvider()

agent = client.as_agent(
    name="MemoryAgent",
    instructions="You are a friendly assistant.",
    context_providers=[memory],
)

In [4]:
session = agent.create_session()

In [5]:
result = await agent.run("Hello! What's the square root of 9?", session=session)
print(f"Agent: {result}\n")

Agent: Hello! The square root of 9 is 3. By the way, what’s your name?



In [6]:
result = await agent.run("My name is Alice", session=session)
print(f"Agent: {result}\n")

Agent: Nice to meet you, Alice! How can I assist you today?



In [7]:
result = await agent.run("What is 2 + 2?", session=session)
print(f"Agent: {result}\n")

Agent: 2 + 2 is 4, Alice!



In [8]:
print(f"[Memory] Stored user name: {memory.user_name}")

[Memory] Stored user name: Alice
